In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_da93cd35edf241fd8ef89ecd453efab8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bsnqTu9d4czY4_6b7SiyTuJlZ2sIPxlkKFKHrPIPXcme',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_da93cd35edf241fd8ef89ecd453efab8.get_object(Bucket='findduplicates-donotdelete-pr-5rzrvcviraltp1',Key='Duplicates_2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

missing_value_formats = ["n.a.","?","NA","n/a","na","--"]
students_df = pd.read_csv(body, na_values=missing_value_formats)
students_df.head()


,SCID,student_district_id,fname,lname,mname,full_name,dob
0,44894,643189.0,JOSIAH,A'FORTIERI,GARRICK BENNETT,JOSIAH A'FORTIERI GARRICK BENNETT,9/11/2006
1,71626,681692.0,MEGAN,AAB,NICOLE,MEGAN AAB NICOLE,4/20/2003
2,37189,636581.0,RANEE,AAMER,ESSMAT,RANEE AAMER ESSMAT,2/14/2007
3,72115,683350.0,CAMRIE,AARON,NaN,CAMRIE AARON,5/18/2014
4,63617,673423.0,CONNOR,AARON,JAMES,CONNOR AARON JAMES,9/19/2001


In [3]:
from fuzzywuzzy import fuzz


def same_student_district_id(r1, r2):
    return r1['student_district_id'] == r2['student_district_id']

def same_dob(r1, r2):
    return r1['dob'] == r2['dob']

def same_full_name(r1, r2):
    return fuzz.ratio(r1['full_name'], r2['full_name']) > 75

def similar_full_name(r1, r2):
    return fuzz.partial_ratio(r1['full_name'], r2['full_name']) > 50

def similar_dob(r1, r2):
    return fuzz.partial_ratio(r1['dob'], r2['dob']) > 50

def same_student(r1, r2):
    return (
        (
            (
                same_student_district_id(r1, r2) and
                similar_full_name(r1, r2)
            ) or
            (
                similar_dob(r1, r2) and
                similar_full_name(r1, r2)
            )
        ) 
    )

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import numpy as np

def find_partitions(df, match_func, max_size=None, block_by=None):
    """Recursive algorithm for finding duplicates in a DataFrame."""

    # If block_by is provided, then we apply the algorithm to each block and
    # stitch the results back together
    if block_by is not None:
        blocks = df.groupby(block_by).apply(lambda g: find_partitions(
            df=g,
            match_func=match_func,
            max_size=max_size
        ))

        keys = blocks.index.unique(block_by)
        for a, b in zip(keys[:-1], keys[1:]):
            blocks.loc[b, :] += blocks.loc[a].iloc[-1] + 1

        return blocks.reset_index(block_by, drop=True)

    def get_record_index(r):
        return r[df.index.name or 'index']

    # Records are easier to work with than a DataFrame
    records = df.to_records()

    # This is where we store each partition
    partitions = []

    def find_partition(at=0, partition=None, indexes=None):

        r1 = records[at]

        if partition is None:
            partition = {get_record_index(r1)}
            indexes = [at]

        # Stop if enough duplicates have been found
        if max_size is not None and len(partition) == max_size:
            return partition, indexes

        for i, r2 in enumerate(records):

            if get_record_index(r2) in partition or i == at:
                continue

            if match_func(r1, r2):
                partition.add(get_record_index(r2))
                indexes.append(i)
                find_partition(at=i, partition=partition, indexes=indexes)

        return partition, indexes

    while len(records) > 0:
        partition, indexes = find_partition()
        partitions.append(partition)
        records = np.delete(records, indexes)

    return pd.Series({
        idx: partition_id
        for partition_id, idxs in enumerate(partitions)
        for idx in idxs
    })


In [5]:
#MAYBE this step is not necessary. Maybe ID doesn't have to be integer. 
students_df = students_df.fillna(0)
students_df['student_district_id'] = students_df['student_district_id'].astype('int')
students_df['student_district_id'] = students_df['student_district_id'].astype('str')

In [6]:
students_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99000 entries, 0 to 98999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SCID                 99000 non-null  int64 
 1   student_district_id  99000 non-null  object
 2   fname                99000 non-null  object
 3   lname                99000 non-null  object
 4   mname                99000 non-null  object
 5   full_name            99000 non-null  object
 6   dob                  99000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [ ]:
students_df['real_id'] = find_partitions(
    df=students_df,
    match_func=same_student
)